![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1173320%2F57353fe4130fbcc013272cc1313112c6%2F1.png?generation=1609000146483661&alt=media)
###SHAP Values - Explicando os impactos das variáveis em suas previsões

Detectar anomalias de dados é um desafio constante, nesse "**Kernelicle**' _(kernel + artigo)_, tentarei dar uma pequena exposição ao [**SHAP**](https://shap.readthedocs.io/en/latest/), um framework especializado em explicações aditivas aplicadas para modelos supervisionados.

Vamos fazer um tour pela visualização de dados e construção de modelo  usando o dataset [**Titanic - Machine Learning from Disaster**](https://www.kaggle.com/c/titanic) por meio deste kernel. 

Se você gosta deste trabalho, por favor, mostre seu apoio por votos positivos.

**Feliz Kaggling!** 🖖😊

_____

##### **Conteúdo**
0. [Etapas em resumo, para irmos direto ao ponto!](#p1) 😏
> * _Data Analysis_
> * _Feature Engineering_
> * _Data Clean_
> * _Preprocessing_
> * _Model_
1. [SHAP](#p2)
>  1. [KernelExplainer](#p3)
>  1. [TreeExplainer](#p4)
2. [Referências](#p5)



<a id="p1"></a>
# 0. Etapas em resumo para seguirmos direto ao ponto!
> _Data Analysis,Feature Engineering, Data Clean, Preprocessing e Model.._
<br />
<img src="https://memegenerator.net/img/images/16435405.jpg" style="height:90px" height="90" />
<hr />

<i>Como esse não é o foco, decidi encurtar as coisas...</i>

In [ ]:
import numpy as np
import pandas as pd
import shap
from sklearn.ensemble import RandomForestClassifier
data = pd.read_csv('../input/titanic-machine-learning-from-disaster/train.csv')
print('missing...')
dfnulls = data.isnull().sum()
cols_isnull = list(dfnulls[dfnulls.values>0].index)
print(dfnulls[dfnulls.values>0])

In [ ]:
data.loc[data.Sex=='male', 'Sex'] = 1
data.loc[data.Sex=='female', 'Sex'] = 2
data.Sex= pd.to_numeric(data.Sex)

data.Age.fillna(value=data.Age.median() ,inplace=True)

data.Fare.fillna(data.Fare.describe().loc['50%'], inplace = True) 
data.Fare = data.Fare.astype('float')

data.Embarked.fillna(value='S', inplace=True)
embark = pd.get_dummies(data.Embarked)
data = pd.concat([data,embark],axis=1)

data.Cabin.fillna(value='U', inplace=True)
data.Cabin = pd.Series([i[0] if not pd.isnull(i) else 'U' for i in data.Cabin ])
data.Cabin  = data.Cabin .map({"U":0, "A":1, "B" : 2 , "C":3, "D":4, "E":5, "F":6, "G":7,"T":0})
data.Cabin  = data.Cabin .astype(int)

data.Ticket = data.Ticket.apply(lambda x: x[:3].strip())
data.Ticket = data.Ticket.astype('category')
data.Ticket = data.Ticket.cat.codes

data["Family"]=-1
data.Family = data.SibSp + data.Parch + 1
data.loc[(data['Family']==1),'Family'] =0
data.loc[(data['Family']>1),'Family'] =1

dropColumns =['Name','SibSp','Parch','PassengerId','Embarked']
data.drop(columns=dropColumns, inplace=True)
y = data.Survived

In [ ]:
data.tail(1)

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
x = data.drop(labels = "Survived", axis = 1)
y = data.Survived
print(x.shape, y.shape)

*Contruindo um model em árvore para verificar quais variáveis são as mais importantes*

<img src="https://instagram.fsdu6-1.fna.fbcdn.net/v/t51.2885-15/e35/105404732_609825429911783_742267563808032849_n.jpg?_nc_ht=instagram.fsdu6-1.fna.fbcdn.net&_nc_cat=111&_nc_ohc=4K2z5Oxx_mIAX8JUtax&tp=1&oh=41d867aa0932b78b070e8990aa23a17c&oe=6011B3BA" style="height:200px" />


In [ ]:
random_state = 3
max_depth = 5
n_estimators = 8 
import matplotlib.pyplot as plt
#em Fibonacci  we trust eheheh
rfc = RandomForestClassifier(max_depth=max_depth, random_state=random_state, n_estimators=n_estimators)
rfc.fit(x, y)
importances = rfc.feature_importances_
indices = np.argsort(importances)
features = x.columns
plt.figure(figsize=(10,5))
plt.title('Features +importante')
plt.barh(range(len(indices)), importances[indices], color='g', align='center',linestyle="solid",alpha=0.8)
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Importância')
plt.show()

<a id="p2"></a>
# 1. SHAP

**SHAP** (SHapley Additive exPlanations) é uma técnica usada para interpretar os "black-box models" para explicar a saída de qualquer modelo de Machine Learning.

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1173320%2F4d39e3a335b20b0bb8e25d58f0e00337%2F1.png?generation=1609003097648061&alt=media) 

Ele conecta a alocação de crédito ideal com explicações locais usando os valores clássicos de Shapley da teoria dos jogos e suas extensões relacionadas ([ver artigos para detalhes e citações](https://github.com/slundberg/shap#citations)).

**Desenvolvido por** [Scott M. Lundberg](https://scottlundberg.com/).


**O SHAP mede o impacto das variáveis, levando em consideração a interação com outras variáveis.**  
*Os valores de Shapley calculam a importância de um recurso comparando o que um modelo prevê com e sem o recurso. No entanto, como a ordem na qual um modelo vê recursos pode afetar suas previsões, isso é feito em todas as ordens possíveis, para que os recursos sejam comparados de maneira justa.*  
[fonte](https://medium.com/@gabrieltseng/interpreting-complex-models-with-shap-values-1c187db6ec83)

![](https://meichenlu.com/img/SHAP_Clustering_XGB.png)

<hr />

**Quais são as vantagens?**
> * **Interpretabilidade global** 
>> Os SHAP Values podem mostrar o quanto cada preditor contribui, positiva ou negativamente, para a variável de destino. 
>> É como o gráfico de importância da variável, mas é capaz de mostrar a relação positiva ou negativa de cada variável com o destino (consulte os gráficos de resumo abaixo).
> * **Interpretabilidade local** 
>> Cada observação obtém seu próprio conjunto de SHAP Values. 
>> Isso aumenta muito sua transparência.  



<a id="p3"></a>
# 1.1 KernelExplainer

O KernelExplainer constrói uma regressão linear ponderada usando os dados de treinamento. Ele calcula os valores de importância de cada feature com base nos valores de Shapley e os coeficientes de uma regressão linear local.
  
> **Desvantagem:** eu longo tempo de execução.

Essa foi a razão de usar um número pequeno na amostragem abaixo! 😅

In [ ]:
explainer = shap.KernelExplainer(rfc.predict_proba,x[:100])
shap_values = explainer.shap_values(x[:100])
shap.summary_plot(shap_values, x[:100])

_A Demora ao executar a celula acima  nos da uma visão unica da força bruta do Kernel SHAP que enumera todo o espaço amostral._ 

💥⛏️ Observe que KernelExplainer faz uma aproximação de amostragem para o valores.

A importancia das features (variáveis) por sua ordenação decrescente. 
_(mais acima, mais importante)_
> * *Class 0* = Morto
> * *Class 1* = Sobrevivente


<a id="p4"></a>
# 1.2 TreeExplainer

O TreeExplainer foi otimizado para renderizar de forma mais eficar modelos em árvore. Como nosso modelo é baseado em **árvore**, o mais apropriado seria usar o TreeExplainer. 😏

> **Nota**: _Em caso de modelos profundos, ele dispões do  DeepExplainer._

In [ ]:
explainer = shap.TreeExplainer(rfc)
shap_values = explainer.shap_values(x)
shap.summary_plot(shap_values, x)

In [ ]:
shap.summary_plot(shap_values[1], x)
shap.summary_plot(shap_values[0], x)


**Observação:**
Quando comparado com a saída de nosso modelo random forest, o grafico de resumo, mostra a mesma classificação de variável para as primeiras quatro variáveis, mas difere para as demais variáveis.

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1173320%2F7a36de4343025b64c1f27a060301d63f%2F1.png?generation=1609005407405063&alt=media)

**Gráfico de dependência**

O gráfico de dependência é um gráfico de dispersão que mostra o efeito que um único feature (variável) em suas previsões.

> * Cada ponto é uma única previsão (linha) em nosso dataset.
> * O eixo x é o valor da feature (nosso xtest).
> * O eixo y é o valor SHAP para a feature especificada.


In [ ]:
shap.dependence_plot("Age", shap_values[1], x)

In [ ]:
shap.dependence_plot('Ticket', shap_values[1], x, interaction_index="Pclass")

⬆️ O gráfico de dependência acima,  nos mostra que existe uma tendência aproximadamente linear e positiva entre **Age** e **Sex**.


**Gráfico de força coletiva**

Cada observação tem seu próprio gráfico de força. 
Se todos os gráficos de força são combinados, girados 90 graus e empilhados horizontalmente.




In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[0], x)

**Gráfico de força individual**

In [ ]:
shap.initjs()
columIndex= 2
shap.force_plot(explainer.expected_value[1], shap_values[1][columIndex,:], x.iloc[columIndex,:], link="logit")

**Gráfico de decisão**
* O eixo x representa a saída do modelo. Nesse caso, as unidades são probabilidades de log.
* O gráfico é centralizado no eixo x em explainer.expected_value
>Todos os valores de SHAP são relativos ao valor esperado do modelo, como os efeitos de um modelo linear são relativos à interceptação.
* Na parte inferior do gráfico, as observações convergem em explainer.expected_value.

In [ ]:
shap.decision_plot(explainer.expected_value[1], shap_values[1], x,link="logit",highlight=1)

**Fim**  

*SHAP é uma ferramenta poderosa na  exploração de padrões que um algoritmo de aprendizadoidentificou*

<a id="p5"></a>
# Referências

* https://www.mdeditor.tw/pl/paB9
* https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values
* https://github.com/slundberg/shap#citations
* https://www.kaggle.com/cast42/feature-importance-and-dependence-plot-with-shap
* https://shap.readthedocs.io/en/latest/overviews.html